In [3]:
import os
import numpy as np
import pandas as pd
import random, tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image

import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
from torch.utils.data import Dataset , DataLoader

In [ ]:
df = pd.read_csv('../input/deepglobe-land-cover-classification-dataset/metadata.csv')
df.head()


In [ ]:
DATA_DIR = '../input/deepglobe-land-cover-classification-dataset'
class_dict = pd.read_csv(os.path.join(DATA_DIR,'class_dict.csv'))
class_names = class_dict['name'].tolist()
class_rgb_values = class_dict[['r','g','b']].values.tolist()
select_class_indices = [class_names.index(cls.lower()) for cls in class_names]
select_class_rgb_values = np.array(class_rgb_values)[select_class_indices]
print(class_names)
select_class_rgb_values

In [ ]:
#helper function for data visualization

def visualize(**images):
    n_images = len(images)
    plt.figure(figsize=(20,8))
    for idx , (name,image) in enumerate(images.items()):
        plt.subplot(1,n_images,idx+1)
        plt.xticks([])
        plt.yticks([])
        plt.title(name.replace('_',' ').title(),fontsize=20)
        plt.imshow(image)
        
    plt.show()
    
    
#perform one hot encoding on the label
def one_hot_encode(label,label_values):
    
    semantic_map = []
    for color in label_values:
        eq = np.equal(label,color)
        class_map = eq.all(axis=-1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map,axis=-1)
    
    return semantic_map

#perform reverse one-hot-encoding on labels/preds
def reverse_one_hot(image,axis):
    
    x = np.argmax(image,axis=axis)
    return x



def color_code_segmentation(image,label_values):
    color_codes = label_values
    x = color_codes[image.astype('int')]
    
    return x


In [ ]:
class LandCoverDataset(Dataset):
    def __init__(self,is_train_valid,csv_file_path,root_dir,class_rgb_values=None,transform=None):
        self.meta_df = pd.read_csv(csv_file_path)
        self.meta_df = self.meta_df.dropna(subset=['split'])
        self.meta_df = self.meta_df.reset_index(drop=True)
        self.root_dir = root_dir
        self.class_rgb_values = class_rgb_values
        self.is_train_valid = is_train_valid
        if is_train_valid == 'train':
            img_paths = self.meta_df[(self.meta_df.split == 'train')]['sat_image_path']
            mask_paths = self.meta_df[(self.meta_df.split == 'train')]['mask_path']
            
        elif is_train_valid == 'val':
            img_paths = self.meta_df[(self.meta_df.split == 'valid')]['sat_image_path']
            
        else:
            img_paths = self.meta_df[(self.meta_df.split == 'test')]['sat_image_path']
        
        #print(mask_paths)
        self.img_path = [os.path.join(self.root_dir,img_p) for img_p in img_paths]
        if is_train_valid == 'train':
            self.mask_path = [os.path.join(self.root_dir,mask_p) for mask_p in mask_paths]
        self.transform = transform
        
    def __len__(self):
        return len(self.img_path)
    
    def __getitem__(self,idx):
        img_path = self.img_path[idx]
        img = cv2.cvtColor(cv2.imread(img_path),cv2.COLOR_BGR2RGB)
        if self.is_train_valid == 'train':
            mask_path = self.mask_path[idx]
            mask = cv2.cvtColor(cv2.imread(mask_path),cv2.COLOR_BGR2RGB)
        
            #one-hot-encode the mask
            mask = one_hot_encode(mask,self.class_rgb_values).astype('float')
        
            if self.transform is not None:
                augmentations = self.transform(image=img,mask=mask)
                img = augmentations['image']
                mask = augmentations['mask']
            
            return img , mask
        
        elif self.is_train_valid == 'test':
            if self.transform is not None:
                img_ten = self.transform(image=img)
                img = img_ten['image']
            return img

In [ ]:


dataset = LandCoverDataset('train','../input/deepglobe-land-cover-classification-dataset/metadata.csv','../input/deepglobe-land-cover-classification-dataset',class_rgb_values=select_class_rgb_values)
random_idx = random.randint(0,len(dataset)-1)
img , mask = dataset[11]

print(len(dataset))
visualize(
    original_image = img,
    ground_truth_mask = color_code_segmentation(reverse_one_hot(mask,axis=-1),select_class_rgb_values),
    one_hot_encoded_mask = reverse_one_hot(mask,axis=-1)
    
)

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2


train_transform = A.Compose([
    A.RandomCrop(height=1024,width=1024,always_apply=True),
    A.Rotate(limit=35, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.1),
    A.Normalize(
        mean=[0.0, 0.0, 0.0],
        std=[1.0, 1.0, 1.0],
        max_pixel_value=255.0,
    ),
    ToTensorV2(),
])

test_transform = A.Compose([
    A.RandomCrop(height=1024,width=1024),
    ToTensorV2(),
])

In [ ]:
aug_dataset = LandCoverDataset('train','../input/deepglobe-land-cover-classification-dataset/metadata.csv','../input/deepglobe-land-cover-classification-dataset',class_rgb_values=select_class_rgb_values,transform = train_transform)
test_dataset = LandCoverDataset('test','../input/deepglobe-land-cover-classification-dataset/metadata.csv','../input/deepglobe-land-cover-classification-dataset/',class_rgb_values=select_class_rgb_values,transform = test_transform)
random_idx = random.randint(0,len(dataset)-1)

print(len(aug_dataset),len(test_dataset))

for idx in range(2):
    img , mask = aug_dataset[idx]
    img = img.permute(1,2,0).numpy().astype('float32')
    mask = mask.numpy().astype('float32')
    #print(color_code_segmentation(reverse_one_hot(mask),select_class_rgb_values).shape)
    visualize(
    original_image = img,
    ground_truth_mask = color_code_segmentation(reverse_one_hot(mask,axis=-1),select_class_rgb_values),
    one_hot_encoded_mask = reverse_one_hot(mask,axis=-1)
    
    )
    

In [ ]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader

train_size = int(0.9*len(aug_dataset))
val_size = len(dataset) - train_size

train_data , val_data = random_split(aug_dataset,[train_size,val_size])

In [ ]:

#loaders
train_loader = DataLoader(train_data,batch_size=2,shuffle=True,num_workers=2,pin_memory=True)
val_loader = DataLoader(val_data,batch_size=1,num_workers=2,pin_memory=True)

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self,in_channels,out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels,out_channels,3,1,1,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels,out_channels,3,1,1,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self,x):
        return self.conv(x)

In [ ]:
class UNET(nn.Module):
    def __init__(self,in_channels=3,out_channels=1,features=[64,128,256,512]):
        super().__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2,stride=2)

        for feature in features:
            self.downs.append(DoubleConv(in_channels,feature))
            in_channels = feature

        self.bottleneck = DoubleConv(features[-1],features[-1]*2)

        for feature in reversed(features):
            self.ups.extend([
                           nn.ConvTranspose2d(feature*2,feature,kernel_size=2,stride=2),
                           DoubleConv(feature*2,feature)
              ])


        self.final_conv = nn.Conv2d(features[0],out_channels,kernel_size=1)

    def forward(self,x):

        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)

        skip_connections = skip_connections[::-1]

        for idx in range(0,len(self.ups),2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]
            if x.shape != skip_connection.shape:
                x = tf.resize(x,size=skip_connection.shape[2:])
            concat_skip = torch.cat((skip_connection,x),dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

In [ ]:
import torch.nn.functional as F

device = ('cuda' if torch.cuda.is_available() else 'cpu')

#set hyperparams
epochs = 5




class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self, weight=None, gamma=2,reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights

    def forward(self, inp, target):

        ce_loss = F.cross_entropy(inp, target,reduction=self.reduction,weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss
    
    
loss_fn = nn.CrossEntropyLoss()


def mIOU(label, pred, num_classes=7):
    pred = F.softmax(pred, dim=1)              
    pred = torch.argmax(pred, dim=1).squeeze(1)
    iou_list = list()
    present_iou_list = list()

    pred = pred.view(-1)
    label = reverse_one_hot(label.cpu(),axis=1).reshape(-1)
    # Note: Following for loop goes from 0 to (num_classes-1)
    # and ignore_index is num_classes, thus ignore_index is
    # not considered in computation of IoU.
    for sem_class in range(num_classes):
        pred_inds = (pred == sem_class)
        target_inds = (label == sem_class)
        if target_inds.long().sum().item() == 0:
            iou_now = float('nan')
        else: 
            intersection_now = (pred_inds[target_inds]).long().sum().item()
            union_now = pred_inds.long().sum().item() + target_inds.long().sum().item() - intersection_now
            iou_now = float(intersection_now) / float(union_now)
            present_iou_list.append(iou_now)
        iou_list.append(iou_now)
    return np.mean(present_iou_list)


model = UNET(in_channels=3,out_channels=7).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=3e-4)



In [ ]:
def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    iou_ = []
    iou = 0.0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.permute(0,3,1,2).to(device)
            mask_pred = model(x.float())
            preds = F.softmax(mask_pred)
            preds = (preds > 0.5).float()
            #print(preds.shape,y.shape)
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / (
                (preds + y).sum() + 1e-8
            )
            iou_.append(mIOU(y,mask_pred))
            del mask_pred
            torch.cuda.empty_cache()
    
    iou = sum(iou_) / len(iou_)
    print(
        f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}"
    )
    print(f"Dice score: {dice_score/len(loader)} , IoU score: {iou} ")
    model.train()
    
    return (dice_score/len(loader)) , iou


def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

In [ ]:
def train_fn(loader,model,optimizer,loss_fn):
    
    loop = tqdm(loader)
    
    loss_ = []
    
    for batch_idx , (data,targets) in enumerate(loop):
        
        data = data.to(device)
        targets = targets.permute(0,3,1,2)
        model.train()
        #forward
        
        preds = model(data)
        loss = loss_fn(preds,reverse_one_hot(targets.long(),axis=1).to(device))
            
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del preds
        
        
        #update tqdm loop
        loop.set_postfix(loss=loss.item())
        
        loss_.append(loss.item())
        torch.cuda.empty_cache()
        
    return sum(loss_) / len(loss_)

        

In [ ]:
from tqdm import tqdm

best_iou_score = 0.0

train_log_list , val_log_list = [] , []


for epoch in range(epochs):
    train_log , val_log = {} , {}
    loss = train_fn(train_loader,model,optimizer,loss_fn)
    train_log['loss'] = loss
    train_log_list.append(train_log)
    

    # check accuracy
    dice_score , iou = check_accuracy(val_loader, model, device=device)
    val_log['dice_score'] = dice_score 
    val_log['IoU'] = iou
    val_log_list.append(val_log)
    
    if best_iou_score < val_log['IoU']:
        best_iou_score < val_log['IoU']
        # save model
        checkpoint = {
            'epoch':epoch,
            "state_dict": model.state_dict(),
            "optimizer":optimizer.state_dict(),
            "loss":loss,
            "dice_score":dice_score,
            "miou":iou
        }
        save_checkpoint(checkpoint)
        print('Model saved!')
        
    

In [ ]:
model = UNET(in_channels=3,out_channels=7)
checkpoint = torch.load('../input/checkpoint/my_checkpoint.pth.tar',map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer'])

In [1]:
for i in range(10):
    
    idx = random.randint(60,100)

    image = test_dataset[idx]
    x_tensor = image.unsqueeze(0)
    # Predict test image
    pred_mask = model(x_tensor.float())
    pred_mask = pred_mask.detach().squeeze().numpy()
    
    # Convert pred_mask from `CHW` format to `HWC` format
    pred_mask = np.transpose(pred_mask,(1,2,0))
    
    # Get prediction channel corresponding to foreground
    pred_mask = color_code_segmentation(reverse_one_hot(pred_mask,axis=-1), select_class_rgb_values)
    
    
    
    visualize(
        original_image = image.numpy().transpose(1,2,0),
        predicted_mask = pred_mask
    )
    
    del image , pred_mask
    
    #torch.cuda.empty_cache()
    

NameError: name 'random' is not defined